In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utils.core_functions import (BUSINESS_DAYS_IN_YEAR,
                                   standardDeviation)
from _utils.portfoliohandcrafiting import (
    create_fx_series_given_adjusted_prices_dict,
    calculate_variable_standard_deviation_for_risk_targeting_from_dict,
    calculate_position_series_given_variable_risk_for_dict,
    calculate_perc_returns_for_dict,
    calculate_perc_returns_for_dict_with_costs,
)
from _utils.strategies.trend_simple_filter import (
    apply_buffering_to_position_dict
)
from _utils.strategies.simple_carry import *


DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [2]:
#%% GET DATA
# # =================

def get_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )
    
    carry_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '_carry.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]   
    )

    return adjusted_prices, current_prices, carry_data

In [3]:
us_adjusted_prices, us_current_prices, us_carry_prices = get_data_dict(['sp500','gas'])
multipliers = dict(sp500=5, gas=10000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(us_adjusted_prices,dict(sp500=1, gas=1))

capital = 2000000
idm = 1.5
instrument_weights = dict(sp500=0.5, gas=0.5)
cost_per_contract_dict = dict(sp500=0.875, gas=15.3)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=us_adjusted_prices,
current_prices=us_current_prices,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

In [ ]:
us_average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

rule_spec = [
        dict(function="carry", span=5),
        dict(function="carry", span=20),
        dict(function="carry", span=60),
        dict(function="carry", span=120),
        dict(function="ewmac", fast_span=16),
        dict(function="ewmac", fast_span=32),
        dict(function="ewmac", fast_span=64),
    ]

us_position_contracts_carry_trend_dict = calculate_position_dict_with_forecast_applied(
        adjusted_prices_dict=us_adjusted_prices,
        carry_prices_dict=us_carry_prices,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=us_average_position_contracts_dict,
        rule_spec=rule_spec,
    )




carry_spans = [5, 20, 60, 120]
    
us_position_contracts_multiple_carry_forecast_dict = (
       calculate_position_dict_with_multiple_carry_forecast_applied(
           adjusted_prices_dict=us_adjusted_prices,
           carry_prices_dict=us_carry_prices,
           average_position_contracts_dict=us_average_position_contracts_dict,
           std_dev_dict=std_dev_dict,
           carry_spans=carry_spans,
       )
   )
fast_spans = [16, 32, 64]
us_position_contracts_multiple_trend_forecast_dict = (
       calculate_position_dict_with_multiple_trend_forecast_applied(
           adjusted_prices_dict=us_adjusted_prices,
           average_position_contracts_dict=us_average_position_contracts_dict,
           std_dev_dict=std_dev_dict,
           fast_spans=fast_spans,
       )
   )


us_position_contracts_fast_trend_forecast_dict = calculate_position_dict_with_trend_forecast_applied(
        adjusted_prices_dict=us_adjusted_prices,
        average_position_contracts_dict=us_average_position_contracts_dict,
        std_dev_dict=std_dev_dict,
        fast_span=16
    )

us_position_contracts_trend_forecast_dict = calculate_position_dict_with_trend_forecast_applied(
        adjusted_prices_dict=us_adjusted_prices,
        average_position_contracts_dict=us_average_position_contracts_dict,
        std_dev_dict=std_dev_dict,
        fast_span=64
    )


us_position_contracts_filter_dict = calculate_position_dict_with_symmetric_trend_filter_applied(
        adjusted_prices_dict=us_adjusted_prices,
        average_position_contracts_dict=us_average_position_contracts_dict,
    )

us_position_contracts_longonly_dict = calculate_position_dict_with_trend_filter_applied(
        adjusted_prices_dict=us_adjusted_prices,
        average_position_contracts_dict=us_average_position_contracts_dict,
    )


#%% calculate returns

us_perc_return_carry_trend_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_carry_trend_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)


us_perc_return_multiple_carry_forecast_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_multiple_carry_forecast_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)


us_perc_return_multiple_trend_forecast_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_multiple_trend_forecast_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)



us_perc_return_fast_trend_forecast_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_fast_trend_forecast_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)


us_perc_return_trend_forecast_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_trend_forecast_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)

us_perc_return_filter_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_filter_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)

us_perc_return_longonly_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_longonly_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)


us_port_carry_trend_return = aggregate_returns(us_perc_return_carry_trend_dict)
us_port_carry_trend_return.sort_index(inplace=True)

us_port_multiple_carry_forecast_return = aggregate_returns(us_perc_return_multiple_carry_forecast_dict)
us_port_multiple_carry_forecast_return.sort_index(inplace=True)

us_port_multiple_trend_forecast_return = aggregate_returns(us_perc_return_multiple_trend_forecast_dict)
us_port_multiple_trend_forecast_return.sort_index(inplace=True)

us_port_fast_trend_forecast_return = aggregate_returns(us_perc_return_fast_trend_forecast_dict)
us_port_fast_trend_forecast_return.sort_index(inplace=True)

us_port_trend_forecast_return = aggregate_returns(us_perc_return_trend_forecast_dict)
us_port_trend_forecast_return.sort_index(inplace=True)


us_port_filter_return = aggregate_returns(us_perc_return_filter_dict)
us_port_filter_return.sort_index(inplace=True)

us_port_longonly_return = aggregate_returns(us_perc_return_longonly_dict)
us_port_longonly_return.sort_index(inplace=True)





In [14]:
# --- Plot retorno acumulado
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x=us_port_carry_trend_return.index,
        y=us_port_carry_trend_return.cumsum()*100,
        name="Carry & Trend",
        line=dict(color="purple"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_multiple_carry_forecast_return.index,
        y=us_port_multiple_carry_forecast_return.cumsum()*100,
        name="Multiple Carry Forecast",
        line=dict(color="grey"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_multiple_trend_forecast_return.index,
        y=us_port_multiple_trend_forecast_return.cumsum()*100,
        name="EWMAC with Multiple Trend Forecast",
        line=dict(color="orange"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_fast_trend_forecast_return.index,
        y=us_port_fast_trend_forecast_return.cumsum()*100,
        name="EWMAC with Fast Trend Forecast",
        line=dict(color="green"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_trend_forecast_return.index,
        y=us_port_trend_forecast_return.cumsum()*100,
        name="EWMAC with Slow Trend Forecast",
        line=dict(color="black"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_filter_return.index,
        y=us_port_filter_return.cumsum()*100,
        name="EWMAC Portfolio - Long Short",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_longonly_return.index,
        y=us_port_longonly_return.cumsum()*100,
        name="EWMAC Portfolio - Long Only",
        line=dict(color="red"),
    )
)



fig.update_layout(
    title="Comparison of Advanced Futures Trading Strategies",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=0.6,
        xanchor="left",
        x=0),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio Carry and Trend Frequência Anual")
print(calculate_stats(us_port_carry_trend_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Carry Trade com Multiplos Carrys Frequência Anual")
print(calculate_stats(us_port_multiple_carry_forecast_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC com filtro de Multiplas Trends Frequência Anual")
print(calculate_stats(us_port_multiple_trend_forecast_return, freq=YEAR))
print("Estatísticas Portfolio EWMAC com filtro de Fast Trend Frequência Anual")
print("================================")
print(calculate_stats(us_port_fast_trend_forecast_return, freq=YEAR))
print("Estatísticas Portfolio EWMAC com filtro de Slow Trend Long Frequência Anual")
print(calculate_stats(us_port_trend_forecast_return,freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC Long and Short na Frequência Anual")
print(calculate_stats(us_port_filter_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC Long Only Frequência Anual")
print(calculate_stats(us_port_longonly_return,freq=YEAR))



Estatísticas de Desempenho
Estatísticas Portfolio Carry and Trend Frequência Anual
{'ann_mean': 19.24746222852722, 'ann_std': 3.2588010695837255, 'sharpe': 5.906301678913424, 'skew': 0.11511732363541796, 'avg_drawdown': 0.11510248088281619, 'max_drawdown': 0.6298956954888829, 'quant_lower': 0.9495722139127037, 'quant_upper': 1.105382099357178}
Estatísticas Portfolio Carry Trade com Multiplos Carrys Frequência Anual
{'ann_mean': 5.197253797978216, 'ann_std': 4.402863403334376, 'sharpe': 1.1804258551474098, 'skew': 0.2148554280757974, 'avg_drawdown': 0.7275255460696698, 'max_drawdown': 1.900445265822802, 'quant_lower': 0.7559661023550914, 'quant_upper': 0.9643945557239347}
Estatísticas Portfolio EWMAC com filtro de Multiplas Trends Frequência Anual
{'ann_mean': 29.291114446777744, 'ann_std': 2.9990557651188143, 'sharpe': 9.766778860018066, 'skew': 0.7396011397339441, 'avg_drawdown': 0.027475027192676256, 'max_drawdown': 0.21262323065899147, 'quant_lower': 0.6197447059653288, 'quant_upper